In [29]:
%%configure -f
{"conf": {"spark.jars": "/system/jar/simpleHTM.jar,/system/jar/jhealpix.jar"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
34,application_1580142637008_0035,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
34,application_1580142637008_0035,spark,idle,Link,Link,✔


In [30]:
import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar


In [31]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HTMUtils


In [32]:
object HEALPixUtils extends Serializable {
    var hp = new Healpix() with Serializable
    val healpixid: (Double, Double) => Long = hp.ang2pix(_,_)

    val healpixidUDF=udf(healpixid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HEALPixUtils


In [33]:
//val datafile = "/user/hive/warehouse/source_test_parquet"
val datafile = "/user/hive/warehouse/source/source_new"
val old_df = spark.read.parquet(datafile)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datafile: String = /user/hive/warehouse/source/source_new
old_df: org.apache.spark.sql.DataFrame = [id: bigint, chunkid: int ... 86 more fields]


In [34]:
val newDF = old_df.withColumn("htmid",HTMUtils.htmidUDF(old_df("coord_ra"),old_df("coord_decl"))).withColumn("healpixid", HEALPixUtils.healpixidUDF(old_df("coord_ra"), old_df("coord_decl")))

newDF.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

newDF: org.apache.spark.sql.DataFrame = [id: bigint, chunkid: int ... 88 more fields]
+----------------+-------+------------------+------------------+--------------+------+-----------------+-----------------+------------------+------------------+-------------------+-------------------+-------------------+----------------+----------------------------+-------------------------------+-------------------------+----------------------------+------------------+---------------------+-----------------+-----------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+----------------------+-----------------------+----------------------+-----------------------+------------------+------------------+-------------------+-----------------+--------------------+--------------------+------------------+--------------------+------------------+----------------+---------------------+---------------------+------------------------+-----

In [ ]:
println("Use MSSQL connector to write to datapool instance ")

val servername = "jdbc:sqlserver://master-0.master-svc"
val dbname = "LSST"
var url = servername + ";" + "databaseName=" + dbname + ";"


val user = "admin"
val password = "fooRiuzg54"



val datapool_table = "dp.Source"

val datasource_name = "SqlDataPool"

val batchsize = 1000000

val start = Calendar.getInstance().getTime()

try {
  newDF.write 
    .format("com.microsoft.sqlserver.jdbc.spark") 
    .mode("append") 
    .option("url", url) 
    .option("dbtable", datapool_table) 
    .option("user", user) 
    .option("password", password) 
    .option("dataPoolDataSource",datasource_name)
    .option("batchsize",batchsize)
    .save()
} catch {
    case e: Throwable => println("MSSQL Connector write failed: " + e)
}
val end = Calendar.getInstance().getTime()
println(start)
println(end)
//var duration = end - start
//println("duration:" + duration)
print("MSSQL Connector write(append) to data pool external table succeeded")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…